In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
def get_page_content_to_sp(request_url):
    #print(request_url)
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}
    html = requests.get(request_url, headers=headers, timeout=10)
    content = html.text
    #print(content)
    soup = BeautifulSoup(content, 'html.parser', from_encoding='utf-8')
    return soup

def get_page_content(request_url):
    #print(request_url)
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}
    html = requests.get(request_url, headers=headers, timeout=10)
    content = html.text
    return content

def analysis(soup):
    temp = soup.find('div', class_='tslb_b')
    #df = pd.DataFrame(columns=['投诉编号','投诉品牌','投诉车系','投诉车型','问题简述','典型问题','投诉时间','投诉状态'])
    tr_list = temp.find_all('tr')
    #print(tr_list[0])
    columns = [i.string for i in tr_list[0].find_all('th')]
    #print(columns)
    df = pd.DataFrame(columns=columns)
    for tr in tr_list[1:]:
        data = [i.string for i in tr.find_all(['td', 'em']) if (i.find('img')) == None ]
        df.loc[len(df)] = data
    return df

def product_covert(x):
    problem = ''
    for i in x.split(','):
        problem += str(type_dict[i]) + ' ' if i in type_dict  else ''
    return problem
        
base_url = 'http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-0-0-0-0-0-'
type_url = 'http://www.12365auto.com/js/cTypeInfo.js?version=' + time.strftime("%Y%m%d", time.localtime()) 
result = pd.DataFrame()
type_info = get_page_content(type_url).split('=')[1]
type_js = eval(type_info)
type_dict = {}
for i in type_js:
    for j in i['items']:
        type_dict[i['value'] + str(j['id'])] = i['name']+str(':') + j['title']
page_num = 10
for i in range(page_num):
    soup = get_page_content_to_sp(base_url+str(i+1)+'.shtml')
    df = analysis(soup)
    result = result.append(df)
result['典型问题'] = result['典型问题'].apply(lambda x : product_covert(x)) 
result.head()
result.to_csv('车质网数据1.csv',index=False)

,投诉编号,投诉品牌,投诉车系,投诉车型,问题简述,典型问题,投诉时间,投诉状态
0,500023,上汽大众,途观L,2017款 330TSI 两驱 豪华版,上汽大众途观L变速箱机电控制单元损坏,变速器:故障灯亮 变速器:滑阀箱故障,2020-07-23,信息审核
1,500021,东风日产,天籁,2019款 2.0L XL Upper 智行版,东风日产天籁发动机异响前减震器异响,发动机:异响 前后桥及悬挂系统:减震器异响 人员技术:多次返修,2020-07-23,信息审核
2,500013,一汽-大众,速腾,2014款 1.4TSI 自动 豪华型,一汽大众速腾变速箱加速无力急加速异响,变速器:异响,2020-07-23,信息审核
3,500011,东风标致,标致408,2015款 1.2T 自动 豪华版,东风标致408发动机烧机油希望尽快解决,发动机:烧机油,2020-07-23,信息审核
4,500000,上汽大众,途观,2013款 1.8TSI 自动 四驱 豪华型,上汽大众途观天窗漏水导致底板被水浸泡,车身附件及电器:天窗漏水,2020-07-23,信息审核


In [22]:
import requests
from lxml import etree
import pandas as pd
res = requests.get('http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-0-0-0-0-0-1.shtml')
res_elements = etree.HTML(res.text)
table = res_elements.xpath('//table[@class="ar_c ar_c1"]')
table = etree.tostring(table[0], encoding='utf-8').decode()
df = pd.read_html(table, encoding='utf-8', header=0)[0]
df['典型问题'] = df['典型问题'].apply(lambda x : product_covert(x))
df

,投诉编号,投诉品牌,投诉车系,投诉车型,问题简述,典型问题,投诉时间,投诉状态
0,500036,北京现代,领动,2019款 1.4T 双离合 炫动·活力型,北京现代领动变速器换挡异响顿挫明显,变速器:异响 变速器:顿挫,2020-07-23,信息审核
1,500031,吉利汽车,帝豪GS,2016款 1.3T 自动 领尚型 运动版,吉利帝豪GS车门生锈多次喷漆未解决,车身附件及电器:车身生锈,2020-07-23,信息审核
2,500025,广汽丰田,凯美瑞,2019款 2.0G 豪华版 国VI,广汽丰田凯美瑞机油增多有严重汽油味,发动机:机油增多,2020-07-23,信息审核
3,500023,上汽大众,途观L,2017款 330TSI 两驱 豪华版,上汽大众途观L变速箱机电控制单元损坏,变速器:故障灯亮 变速器:滑阀箱故障,2020-07-23,信息审核
4,500021,东风日产,天籁,2019款 2.0L XL Upper 智行版,东风日产天籁发动机异响前减震器异响,发动机:异响 前后桥及悬挂系统:减震器异响 人员技术:多次返修,2020-07-23,信息审核
5,500013,一汽-大众,速腾,2014款 1.4TSI 自动 豪华型,一汽大众速腾变速箱加速无力急加速异响,变速器:异响,2020-07-23,信息审核
6,500011,东风标致,标致408,2015款 1.2T 自动 豪华版,东风标致408发动机烧机油希望尽快解决,发动机:烧机油,2020-07-23,信息审核
7,500000,上汽大众,途观,2013款 1.8TSI 自动 四驱 豪华型,上汽大众途观天窗漏水导致底板被水浸泡,车身附件及电器:天窗漏水,2020-07-23,信息审核
8,499997,东风雪铁龙,雪铁龙C4L,2016款 1.6T 自动 精英型,东风雪铁龙C4L发动机失去动力厂家不保修,发动机:异响 发动机:动力消失 发动机:涡轮增压器故障 服务态度:不予索赔,2020-07-23,厂家受理
9,499995,英菲尼迪（进口）,英菲尼迪QX60,2016款 2.5T Hybrid 两驱 卓越版 国V,进口英菲尼迪QX60变速箱顿挫有异响,变速器:异响 变速器:顿挫,2020-07-23,信息审核
